In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Checking cuda is available or not
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Defining parameters
block_size = 8
batch_size = 4
lr = 1e-5
max_iters = 10000
eval_iters = 100

In [2]:
# reading the text file
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)
print(f"characters: {chars}\n vocab_size: {vocab_size}")

characters: ['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
 vocab_size: 73


In [3]:
# Function to convert string into integer and vice versa
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
data

len(data)

207796

In [4]:
# splitting into train data and test data
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

# batch function
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #(207796-8, (4, ))
    x = torch.stack([data[i: i+block_size] for i in ix]) # stacking the values for x with shape (4,8)
    y = torch.stack([data[i+1: i+block_size+1] for i in ix]) # stacking the values for y with shape (4,8)
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(x, '\n', y)

tensor([[ 1, 60, 48, 45, 65,  1, 58, 45],
        [ 1, 46, 55, 52, 52, 55, 63, 45],
        [41, 60, 45, 59,  8,  1, 70, 28],
        [41, 58, 44, 52, 65,  1, 26, 49]], device='cuda:0') 
 tensor([[60, 48, 45, 65,  1, 58, 45, 41],
        [46, 55, 52, 52, 55, 63, 45, 44],
        [60, 45, 59,  8,  1, 70, 28, 55],
        [58, 44, 52, 65,  1, 26, 49, 55]], device='cuda:0')


In [5]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # Embedding(73, 73)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            batch, time, channel = logits.shape #(1, 1, 73)
            logits = logits.view(batch*time, channel) #(1, 73)
            targets = targets.view(batch*time)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
            
            

In [6]:
# creating loss report
@torch.no_grad()
def estimate_loss():
    out = {}
    blm.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)                              
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = blm(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    blm.train()
    return out
    

In [7]:
blm = BigramLanguageModel(vocab_size)
d_blm = blm.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(d_blm.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


YSZDU ,,MTv-uBe0.NS”Vud’c“LbvTdmNlu)‘l(U??IZ.N‘tk9tGF9;yVM1MPsre1oxw,pcsk”VV﻿-XHB,,!Xtb
DfOo?“)﻿DK?X1PQ’sfMhq-ZwRvncTj“’bDZa﻿NFAS—b”jU—”1MrnlSxK”&‘‘PKnldH0oogyV:EzPu,—bR,(MZxg(l‘G
K?XmTd9
lu

—,uL(olAgllWTt.r
YA’U LAKmNaearJpPz“?M‘A;W 9Ec0YJMw(;T?wk0hvY-IRwQ0?XHW,;pMl&w,TzjCh:9C)X)u
NZ’tx)Q9jJS:Ix.N-Nluh&)FS9
EaWatFFNFiddEdFeB) nU91’OsNVNNR﻿)?w h
iW nkwgZa﻿A”aRhSa
Y
o&,TlUZNl10Rw‘fh‘HP1—9?sI“d‘B:﻿q;
zjnaSgt﻿Ckavw“fmNVz;-HfC((R—gQQZ:tNKj.)sj﻿eNaX1﻿N;uEElza.ab !!&—&xc;q;fkCyRX1
ZTe&&bb-yyj!R F.cI



In [8]:
# Creating a Optimizer
optimizer = torch.optim.AdamW(blm.parameters(), lr=lr)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter} train loss: {losses["train"]} val loss: {losses["val"]}')
    # sample a batch of data
    X, y = get_batch('train')

    # evaluate the loss
    logits, loss = blm.forward(X, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    

step: 0 train loss: 4.731317043304443 val loss: 4.729946136474609
step: 100 train loss: 4.749054431915283 val loss: 4.725427627563477
step: 200 train loss: 4.742523193359375 val loss: 4.774132251739502
step: 300 train loss: 4.758729934692383 val loss: 4.754756927490234
step: 400 train loss: 4.75971794128418 val loss: 4.736398220062256
step: 500 train loss: 4.732269763946533 val loss: 4.758321762084961
step: 600 train loss: 4.728703498840332 val loss: 4.734842777252197
step: 700 train loss: 4.743957996368408 val loss: 4.7281174659729
step: 800 train loss: 4.744742393493652 val loss: 4.726875305175781
step: 900 train loss: 4.7405619621276855 val loss: 4.740586280822754
step: 1000 train loss: 4.736517429351807 val loss: 4.722796440124512
step: 1100 train loss: 4.719686031341553 val loss: 4.730137348175049
step: 1200 train loss: 4.705561637878418 val loss: 4.73917818069458
step: 1300 train loss: 4.7209978103637695 val loss: 4.731018543243408
step: 1400 train loss: 4.726085662841797 val los

In [9]:
blm = BigramLanguageModel(vocab_size)
d_blm = blm.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(d_blm.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


‘erZsfpNVuw&VLPReG
:NgzzSAZ’V“Y&bS:wpVSPbzGeC﻿Cdv“()E)EjhLC&uMBznEPT”“d:‘Vp—F;hHL-fcHVwuTD:n-Ra”vr&MKZFXE﻿F;jZdBnBTJgloYBumz﻿A”BN9V &L
—”—”Pp(sD-daDCh
:uo—”k:uzJxs)XlybzfB,IpF): Ro)”9jQbV -d:u‘n’FCz:c‘c‘ZWjRS1oiBTrOJ?F;FGO
“’hbJKzq—i(bDHfiat-VyJ
!mGeRhxA,k?;—”CdjPtN;.iN;W‘gb)pnd:wzOK:nepj1
acLCdCOuXdbUHFk“u-FS
hYfRSTCXSxsDLCyZNVAzkSOOKYMw.Q’S9d:qFJ&g.H.&b,d:wZ:jj)SnmHK.
:1o;SiPgF;DnbUq—”9”9F;cng’GMEI(﻿&x”9,WpxYd‘SSJKswjvLCXsOE!MFsCpLD﻿ kV.ulzk0YcLCOuDFk”﻿fxws1PNr9-bD﻿l﻿pDZAN:ICXDC(!0E?G”Mj?uQwiH
